<a href="https://colab.research.google.com/github/GheorgheBriana/Big-data-faculty/blob/main/lab3_part2_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master = "local[2]")

In [3]:
#Paralelizati lista primelor 1.000.000 numere intregi
#trebuie sa cream un RDD din lista numerelod [0, 1.000.000)
A = sc.parallelize(range(0, 1000000)) # se creeaza un RDD dintr o colectie locala
# nu se zice cate partitii sa aiba. deoarece am inceput cu un local[2] => creeaza 2 partitii

In [4]:
print(A.getNumPartitions()) # afiseaza numarul de partitii ale RDD-ului A
# Spark imparte datele in partitii care se proceseaza in paralel

2


In [5]:
D = A.repartition(10)
# reimpartim RDD-ul A in 10 partitii

In [6]:
print(D.getNumPartitions()) #vedem ca D are 10 partitii

10


In [7]:
A = sc.parallelize(range(1000000), numSlices = 8) #cream din nou un RDD a cu 1 milion de numere
# spunem de la inceput sa fie impartit in 8 partitii

In [9]:
print(A.getNumPartitions())
#afiseaza 8 partitii

8


In [11]:
A = sc.parallelize(range(1000000)).map(lambda x: (x, x)).partitionBy(10)
# map(lambda x: (x, x) -> transforma fiecare element in tuplu. EX: 0 => (0,0) etc
#partitionBy(10) -> 10 partitii
print(A.glom().map(len).collect())
#glom() -> tranforma fiecare partitie intr o lista de elemente
# map(len) -> calculeaza nr de elemente din fiecare partitie
#collect() -> afiseaza rezultatele

[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]


In [ ]:
#Presupunem că dorim să folosim funcția filter()
# pentru a selecta o parte din elementele lui A
# În unele partiții pot rămâne mai multe elemente decât în altele


In [13]:
#Selectati 20% din elemente, pe baza unui filtru care retine doar numerele divizibile cu 5
#Obtineti numarul de elemente din partitiile lui B
B = A.filter(lambda x: x[0] % 5 == 0)
print(B.glom().map(len).collect())

[100000, 0, 0, 0, 0, 100000, 0, 0, 0, 0]


In [ ]:
#Redistribuiti B in 10 partitii cu numar egal de elemente
#Obtineti numarul de elemente din partitiile lui C

In [17]:
#a) cu partitionBy (controlat, presupune(key, value)
C = B.partitionBy(10)
print(C.glom().map(len).collect())
#elementele s-au dus doar in anumite partitii (nu uniform)
# in acest caz (divizibilitate cu 5), cheile sunt multiplii de 5
# => doar partitiile 0 si 5 sunt umplute, restul raman goale

[100000, 0, 0, 0, 0, 100000, 0, 0, 0, 0]


In [16]:
#b) cu repartition (aleator, fara cheie)
D = B.repartition(10)
print(D.glom().map(len).collect())
# echilibreaza datele in mod egal

[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000]


In [ ]:
#Dorim sa obtinem primul si ultimul element din partitiile neechilibrate
# pe care le-am obtinut din A in B. Partitie vida => NONE

In [ ]:
# Definiți funcția ce returnează tuplul de 3 elemente cerut pentru fiecare partiție
#def getPartitionInfo(P):

#Apelați funcția pentru partițiile lui B și afișați rezultatul


In [19]:
def getPartitionInfo(P):
  if len(P) == 0:
    return None
  else:
    primul = P[0]
    ultimul = P[-1]
    count = len(P)
    suma = primul + ultimul
    return (primul, ultimul, suma)

In [22]:
resultB = B.glom().map(getPartitionInfo).collect()
print(resultB)

[((0, 0), (999990, 999990), (0, 0, 999990, 999990)), None, None, None, None, ((5, 5), (999995, 999995), (5, 5, 999995, 999995)), None, None, None, None]


In [23]:
resultA = A.glom().map(getPartitionInfo).collect()
print(resultA)

[((0, 0), (999990, 999990), (0, 0, 999990, 999990)), ((1, 1), (999991, 999991), (1, 1, 999991, 999991)), ((2, 2), (999992, 999992), (2, 2, 999992, 999992)), ((3, 3), (999993, 999993), (3, 3, 999993, 999993)), ((4, 4), (999994, 999994), (4, 4, 999994, 999994)), ((5, 5), (999995, 999995), (5, 5, 999995, 999995)), ((6, 6), (999996, 999996), (6, 6, 999996, 999996)), ((7, 7), (999997, 999997), (7, 7, 999997, 999997)), ((8, 8), (999998, 999998), (8, 8, 999998, 999998)), ((9, 9), (999999, 999999), (9, 9, 999999, 999999))]
